In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, impute
from IPython.display import display

from sklearn.dummy import DummyRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, cross_validate
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import Ridge, RidgeCV

from fbprophet import Prophet
import optuna
import statsmodels.api as sm
import scipy.stats as stats

plt.style.use('ggplot')

In [1]:
random_state = 42
rng = np.random.RandomState(random_state)

In [1]:
X = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv', index_col=0)
X_test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv', index_col=0)

In [1]:
print(X.columns.difference(X_test.columns))
y = X.pop('pressure')

In [1]:
num_columns = ['R', 'C', 'u_in', 'u_out']
ds = 'time_step'
ts_id = 'breath_id'

# Quick EDA

In [1]:
X[ts_id].unique()

In [1]:
(X.isna().sum() > 0).any(), (X_test.isna().sum() > 0).any()

In [1]:
X[num_columns].describe(), X_test[num_columns].describe()

In [1]:
X[num_columns].nunique()

In [1]:
X.groupby('breath_id')['R'].nunique().max(), X.groupby('breath_id')['C'].nunique().max()

R and C are physical parameters which change at every breath

In [1]:
pd.concat([X.groupby('breath_id')[ds].max().rename('train ts'),
           X_test.groupby('breath_id')[ds].max().rename('test ts')], axis=1)\
    .plot.hist(bins=20, density=True, alpha=0.5)

breath duration is between 2.5s and 2.8s

In [1]:
pd.concat([X.groupby('breath_id')['u_in'].mean().rename('train u_in'),
           X_test.groupby('breath_id')['u_in'].mean().rename('test u_in')], axis=1)\
    .plot.hist(bins=20, density=True, alpha=0.5)

In [1]:
fig, axs = plt.subplots(nrows=3, figsize=(12, 12))

df_combined = pd.concat([X.assign(set='train'),
                         X_test.assign(set='test')], axis=0)

def plot_counts_cst(col, ax, X=X, X_test=X_test):
    df_plot = pd.concat([(X[col].value_counts() / X.shape[0]).rename('ratio train'),
                         (X_test[col].value_counts() / X_test.shape[0]).rename('ratio test')], axis=1)
    return df_plot.plot.bar(ax=ax)

plot_counts_cst('R', axs[0])
plot_counts_cst('C', axs[1])
plot_counts_cst('u_out', axs[2])

In [1]:
ds_diff = X.groupby(ts_id)[ds].transform(lambda x: x.diff())

In [1]:
ds_diff.describe(percentiles=np.arange(0.1, 1.0, 0.1))

In [1]:
issue_ts_id = X.assign(time_diff=lambda x: x['time_step'].diff())\
               .query('time_diff > 0.2 and u_out == 0')\
               [ts_id].unique()

In [1]:
X\
.loc[X[ts_id] == 36175, :]\
.assign(time_diff=lambda x: x[ds].diff(),
        y=y)\
.plot(x='time_step', subplots=True, figsize=(12, 12))

Some issue with the time_step for "breath_id":
24127,  36175,  38415,  46324,  54129,  55244,  55851,  72104, 74766, 104001, 109693, 119689

In [1]:
# plt.hist(y, bins='auto')
plt.hist(y.groupby(X[ts_id]).max(), bins='auto')

plt.show()

# Baseline

In [1]:
def get_folds(X, y, n_splits=5, ts_id=ts_id):
    # return fixed list of CV index (for XGB, LGB compability to groups)
    folds = GroupKFold(n_splits=n_splits)
    return list(folds.split(X, y, groups=X[ts_id]))


def score_model(model, X=X, y=y, folds=None, fit_as_score=True):
    if folds is None:
        folds = get_folds(X, y)
        
    sample_weight = X.mask(lambda x: x['u_out'] == 0).isna().any(axis=1).astype(int)
    
    def scorer(y_true, y_pred, sample_weight=sample_weight):
        return mean_absolute_error(y_true.values, y_pred, sample_weight=sample_weight.loc[y_true.index].values)
    scoring = make_scorer(scorer, greater_is_better=False, sample_weight=sample_weight)
    
    if fit_as_score:
        fit_params = {'sample_weight':sample_weight}
    else:
        fit_params = {}
        
    d = cross_validate(model, X, y, scoring=scoring, cv=folds, n_jobs=1, fit_params=fit_params, return_train_score=True)
    d['train_score'] = -d['train_score']
    d['test_score'] = -d['test_score']
    return pd.DataFrame(d).style.bar(subset=['fit_time', 'score_time'], vmin=0, vmax=30)\
                                .bar(subset=['test_score', 'train_score'], vmin=0, vmax=7.5)

In [1]:
score_model(Ridge(), X=X, y=y)

In [1]:
score_model(Ridge(max_iter=100_000), X=X.loc[X['u_out'] == 0], y=y.loc[X['u_out'] == 0])

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=X, y=y)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=X.loc[X['u_out'] == 0], y=y.loc[X['u_out'] == 0])

# Trial - Processing

In [1]:
columns_discrete = [ts_id, 'u_out', 'R', 'C']

def add_lag(X, k=1, fill=0):
    X = X.copy()
    if isinstance(k, int):
        k = list(range(1, k+1))  
    
    for i in k:
        X_lag = X.drop(columns=columns_discrete).groupby(X[ts_id]).shift(i)
        if fill is not None:
            X_lag = X_lag.fillna(fill)
        X = X.join(X_lag.add_prefix(f'lag_{i}_'))
        
    return X

def proc0(X):
    pi = np.pi
    X = X.query('u_out==0')\
         .eval('''RC=R*C/1000
                  tau=2 * @pi * RC
                  t=time_step/tau
                  et=1-exp(-t)
                  uet=u_in*et''')\
         .assign(time_diff=lambda x: x.groupby(ts_id)['time_step'].diff().fillna(0),
                 int_u_in=lambda x: x.eval('u_in*time_diff').groupby(x[ts_id]).cumsum(),
                 sum_u_in=lambda x: x.groupby(ts_id)['u_in'].cumsum(),
                 diff_u_in=lambda x: x.groupby(ts_id)['u_in'].diff().fillna(0),)
    return X

columns_ts = ['t', 'et', 'uet', 'u_in', 'int_u_in']

X0 = proc0(X)
y0 = y.loc[X['u_out'] == 0]

# EDA (Univariate)

In [1]:
n_rows = 10_000

In [1]:
y_ts = y.set_axis(X[ds]).iloc[:n_rows]
X_ts = X.set_axis(X[ds], axis='index').iloc[:n_rows]

X_ts0 = proc0(X_ts)
X_ts0_lag = add_lag(proc0(X.iloc[:n_rows]))
X_ts0_lag = X_ts0_lag.set_axis(X_ts0_lag[ds], axis='index')

y_ts0 = y_ts.loc[X_ts['u_out'] == 0]

In [1]:
fig, ax = plt.subplots(figsize=(10, 10))

y_ts.groupby(X_ts[ts_id])\
 .plot(style='-k', alpha=0.1, ax=ax)

y_ts.loc[lambda x: x.groupby(X_ts[ts_id]).transform('max') == x]\
.plot(style='.r', ax=ax, alpha=0.4)

In [1]:
from sklearn.feature_selection import mutual_info_regression, f_regression

def show_mi(X, y=y, random_state=random_state, columns_discrete=[]):
    mi = mutual_info_regression(X, y,
                                random_state=random_state,
                                discrete_features=X.columns.isin(columns_discrete))

    s = pd.Series(mi, index=X.columns, name='mutual_info_regression')\
          .sort_values(ascending=False)
          
    return s, s.to_frame().style.bar(vmin=0, vmax=1)


def show_f(X, y=y, center=False):
    f = f_regression(X, y, center=center)[0]

    s = pd.Series(f, index=X.columns, name='f_regression F-score')\
          .sort_values(ascending=False)
          
    return s, s.to_frame().style.bar()

In [1]:
show_mi(X_ts, y_ts, columns_discrete=['u_out'])[1]

In [1]:
show_mi(X_ts0_lag, y_ts0)[1]

In [1]:
show_f(X_ts0_lag, y_ts0, center=True)[1]

In [1]:
show_mi(X_ts0.groupby(ts_id).max(), y_ts0.groupby(X_ts0[ts_id]).max())[1]

In [1]:
show_mi(X_ts0.groupby(ts_id).first(), y_ts0.groupby(X_ts0[ts_id]).first())[1]

In [1]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 10), sharex=False)

for c, ax in zip(['u_out', 'R', 'C', 'RC'], axs.flat):
    if c == 'u_out':
        sns.boxplot(x=X[c], y=y, ax=ax)
    else:
        sns.boxplot(x=X0[c], y=y0, ax=ax)
    
plt.tight_layout()

In [1]:
fig, ax = plt.subplots(figsize=(10, 10))

y_ts0.groupby(X_ts0[ts_id])\
 .plot(style='-k', alpha=0.1, ax=ax)

y_ts0.loc[lambda x: x.groupby(X_ts0[ts_id]).transform('max') == x]\
.plot(style='.r', ax=ax, alpha=0.4)

In [1]:
fig, axs = plt.subplots(nrows=len(columns_ts), figsize=(10, 5*len(columns_ts)), sharex=False)

for c, ax in zip(columns_ts, axs):
    X_ts0[c].groupby(X_ts0[ts_id])\
            .plot(style='-k', alpha=0.1, ax=ax)
    ax.set(ylabel=c)
    
plt.tight_layout()

In [1]:
n_plot = 5

from cycler import cycler
fig, axs = plt.subplots(nrows=n_plot, figsize=(10, 8*n_plot))

breath_ids = X_ts0['breath_id'].drop_duplicates().sample(n_plot, random_state=random_state)

for i, ax in zip(breath_ids, axs, ):
    X_ts0.assign(y=y_ts0)\
         .query(f'breath_id=={i}')[columns_ts+['y']]\
         .plot(secondary_y=['u_in', 'int_u_in', 'y'], ax=ax, style=['--', '--', '--', '-.', ':', 'k-'])
    ax.set_title(i)

In [1]:
to_delete = 'X0, y0, X_ts, y_ts, X_ts0, y_ts0, X_ts0_lag'.split(', ')
for x in to_delete:
    globals().pop(x, None);

# Processing

In [1]:
def add_lag(X, k=1, columns=['u_in', 'time_step'], fill=0):
    X = X.copy()
    if isinstance(k, int):
        k = list(range(1, k+1))
    if columns is None:
        columns = X.columns.tolist()
    
    for i in k:
        X_lag = X[columns].groupby(X[ts_id]).shift(i)
        if fill is not None:
            X_lag = X_lag.fillna(fill)
        X = X.join(X_lag.add_prefix(f'lag_{i}_'))
        
    return X


def proc(X):
    pi = np.pi
    X = X.eval('''RC=R*C/1000
                  tau=2 * @pi * RC
                  t=time_step/tau
                  et=1-exp(-t)
                  uet=u_in*et''')\
         .assign(time_diff=lambda x: x.groupby(ts_id)['time_step'].diff().fillna(0),
                 int_u_in=lambda x: x.eval('u_in*time_diff').groupby(x[ts_id]).cumsum(),
                 sum_u_in=lambda x: x.groupby(ts_id)['u_in'].cumsum(),
                 diff_u_in=lambda x: x.groupby(ts_id)['u_in'].diff().fillna(0),
                 max_u_in=lambda x: x.groupby(ts_id)['u_in'].transform('max'),)
    return X

# Compare to baseline

In [1]:
X_proc = proc(X).query('u_out==0')
y_proc = y.loc[X['u_out'] == 0]

In [1]:
X_proc.isna().any(axis=None)

## With lags

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X), y=y)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X, k=2), y=y)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X, k=3), y=y)

## With processing (and lag)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=X_proc, y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X_proc, columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in']), y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(proc(X), columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in']), y=y)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X_proc, k=2, columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in']), y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X_proc, k=3, columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in']), y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4), X=add_lag(X_proc, k=[2], columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in']), y=y_proc)

# Hyperparameter tuning - manual

In [1]:
X_proc = add_lag(proc(X).query('u_out==0'), k=[2], columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in'])
y_proc = y.loc[X['u_out'] == 0]

# select categorical as 1 value at each "breath_id"
categorical_feature = [ts_id] + (X_proc.groupby(ts_id).nunique().mean() == 1).loc[lambda x: x].index.tolist()
# X_post[categorical_features] = pd.Categorical(X_post[categorical_features])

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4, random_state=random_state), X=X_proc, y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4, objective="regression_l1", random_state=random_state), X=X_proc, y=y_proc)

In [1]:
score_model(lgb.LGBMRegressor(n_jobs=4, objective="regression_l1", random_state=random_state, n_estimators=1_000), X=X_proc, y=y_proc)

## Best "learning_rate"

In [1]:
import optuna.integration.lightgbm as opt_lgb

dtrain = lgb.Dataset(data=X_proc, label=y_proc)

eval_hist = {}
for learning_rate in np.geomspace(1e-1, 1, 10):
    eval_hist[learning_rate] = lgb.cv(
        params={"objective":"regression_l1", "learning_rate":learning_rate, "verbose":-1, 'num_threads':4},
        train_set=dtrain,
        num_boost_round=500,
        early_stopping_rounds=50,
        folds=get_folds(X_proc, y_proc),
        stratified=False,
        shuffle=False,
        metrics='l1',
        verbose_eval=False,
        #eval_train_metric=True,
        seed=random_state,
    )

In [1]:
r_eval_hist = {}
for eta, d in eval_hist.items():
    r_eval_hist[eta] = pd.Series(d['l1-mean'])
    
r_eval_hist = pd.DataFrame(r_eval_hist)
r_eval_hist.plot()

display(pd.concat([r_eval_hist.idxmin().rename('num_boost'),
                   r_eval_hist.min().rename('min_test_score')], axis=1))

# Next steps - Hyperparameter fine-tuning  
[Ventilator Pressure Prediction - LGB Optuna](https://www.kaggle.com/squarex/ventilator-pressure-prediction-lgb-optuna)